## Library Setup

### Transformations
 - <mark>__Scale Down Ordinal Data By Substracting all samples with Global Feature Minimum__</mark>
 - <mark>__Sort Columns by Name for better visibility__</mark>
 - <mark>__Ordinally Encode Nominal Data to reduce categorical distance within a feature__</mark>
 - <mark>__Add a Column - Sum of all Binary Features along x-axis__</mark>
 - <mark>__Standardise the Ratio Features using MinMaxScaler__</mark>
 - <mark>__Group By binary Literal__</mark>
 - <mark>__Generated A Normal Rank for all Ordinal Features__</mark>
 - <mark>__Generated Dummy Variable for categoricals__</mark>
 - <mark>__Made all Type Conversions__</mark>


In [1]:
# import widen_notebook
from mysetup import NotebookFinder
import sys
sys.meta_path.append(NotebookFinder())

from setup_transform import *
%matplotlib inline
cl_weight = sklearn.utils.compute_class_weight(class_weight='balanced',classes=[0,1,2],y=y)
CLASS_WEIGHTS = {i:cl_weight[i] for i in range(3)}    
lgr_params = dict(class_weight=CLASS_WEIGHTS,fit_intercept=True,multi_class='ovr',max_iter=200000,random_state=42,n_jobs=24, penalty ='elasticnet' ,cv=RepeatedStratifiedKFold(n_splits=3,n_repeats=5,random_state=30),scoring='f1_macro',solver='saga',Cs=100,l1_ratios= np.linspace(0,1,endpoint=False,num=10))
def_cv = RepeatedStratifiedKFold(n_splits=3,n_repeats=2,random_state=42)
from sklearnex import unpatch_sklearn
unpatch_sklearn()
import copy
import warnings
warnings.filterwarnings('ignore')

importing Jupyter notebook from setup_transform.ipynb


/home/u164131/.conda/envs/ml/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


## Training